# [Palindrome Free Strings](https://codingcompetitions.withgoogle.com/kickstart/round/00000000008cb33e/00000000009e762e#problem)
- **Given**: a string `S` size `N`
    + Consisting of characters `0`, `1`, and `?`
    + `?` can be replaced by `0` or `1`
- **Find**: if it is `POSSIBLE` if existing a string has **no substrings that are palindromes of length 5 or more**

#### Constraints
- $1 \leq N \leq 5 \times 10^4$

#### Examples

```
// Input
100???001

// Output
IMPOSSIBLE
```

```
// Input
100??

// Output
POSSIBLE

// Explain
existing a string 10011 satisfy the requirement
```


# Solution
#### Insights
- Palindrome
    + If string size = `odd`: If substring size = 5 satify the requirement -> 7,9,11,etc. also satisfy
    + If string size = `even`: only need to check substring size = 6 -> 8,10,12,etc. also satisfy
    + $\to$ Only need to check palindrome of substring size 5 and 6
- Optimize a complete search: left -> right
    + dp: only keep the last 5 character info
    + cache: a string size 6 if s is not palindrome size = 5 and size = 6

#### topdown + cache

```Cpp
int N;
string S;



// cache[s]: if s is not palindrome size = 5 and size = 6
unordered_map<string, bool> cache;
bool is_pal(const string &s) {
    int l = 0;
    int r = s.size() -1;
    while(l < r) {
        if(s[l] != s[r]) return false;
        l += 1;
        r -= 1;
    }
    return true;
}
bool check_next(const string &s, char c) {
    string next_str = s + c;

    // next string size < 5
    if(next_str.size() < 5) return true;

    // next string size = 5
    if(next_str.size() == 5) {
        if(cache.count(next_str)) return cache[next_str];

        // check palindrome size 5
        if(is_pal(next_str)) return cache[next_str] = false;
        return cache[next_str] = true;
    }

    // next string size = 6
    if(next_str.size() == 6) {
        if(cache.count(next_str)) return cache[next_str];

        // check palindrome size 6
        if(is_pal(next_str)) return cache[next_str] = false;

        // check palindrome size 5
        string tmp = next_str.substr(1);
        if(is_pal(tmp)) return cache[next_str] = false;

        return cache[next_str] = true;
    }
    assert(false);
}

unordered_set<string> dp[50003];
void dfs(int idx) {
    // base cases
    if(idx == 0) {
        if(S[idx] == '?') dp[idx] = {"1", "0"};
        else if(S[idx] == '0') dp[idx] = {"0"};
        else if(S[idx] == '1') dp[idx] = {"1"};
        return;
    }

    // top down
    dfs(idx - 1);

    // get state[idx] from idx-1
    if(S[idx] == '?') {
        for(const string &s: dp[idx-1]) {
            if(check_next(s , '1')) {
                // only keep the last 5 character
                string tmp = s + '1';
                tmp.size() == 6 ? dp[idx].insert(tmp.substr(1)) : dp[idx].insert(tmp);
            }
            if(check_next(s , '0')) {
                // only keep the last 5 character
                string tmp = s + '0';
                tmp.size() == 6 ? dp[idx].insert(tmp.substr(1)) : dp[idx].insert(tmp);
            }
        }
    } else {
        for(const string &s: dp[idx-1]) {
            if(check_next(s, S[idx])) {
                // only keep the last 5 character
                string tmp = s + S[idx];
                tmp.size() == 6 ? dp[idx].insert(tmp.substr(1)) : dp[idx].insert(tmp);
            }
        }
    }
}


string sol() {
    // cache.clear();
    for (int i=0; i<N; ++i) dp[i].clear();

    dfs(N-1);
    return (dp[N-1].size() > 0) ? "POSSIBLE" : "IMPOSSIBLE";
}
```

#### Bottomup + cache

```C++
int N;
string S;


unordered_set<string> dp;
void generate(int idx,const string &cur) {
    if(idx == 4) {
        dp.insert(cur);
        return;
    }

    if(S[idx] == '?') {
        generate(idx+1, cur + '1');
        generate(idx+1, cur + '0');
    } else {
        generate(idx+1, cur + S[idx]);
    }
}

// cache[s]: if s is not palindrome size = 5 and size = 6
unordered_map<string, bool> cache;
bool is_pal(const string &s) {
    int l = 0;
    int r = s.size() -1;
    while(l < r) {
        if(s[l] != s[r]) return false;
        l += 1;
        r -= 1;
    }
    return true;
}
bool check_next(const string &s, char c) {
    string next_str = s + c;

    // next string size < 5
    if(next_str.size() < 5) return true;

    // next string size = 5
    if(next_str.size() == 5) {
        if(cache.count(next_str)) return cache[next_str];

        // check palindrome size 5
        if(is_pal(next_str)) return cache[next_str] = false;
        return cache[next_str] = true;
    }

    // next string size = 6
    if(next_str.size() == 6) {
        if(cache.count(next_str)) return cache[next_str];

        // check palindrome size 6
        if(is_pal(next_str)) return cache[next_str] = false;

        // check palindrome size 5
        string tmp = next_str.substr(1);
        if(is_pal(tmp)) return cache[next_str] = false;

        return cache[next_str] = true;
    }
    assert(false);
}


unordered_set<string> get_next_state(const unordered_set<string> dp, char c) {
    unordered_set<string> dp_next;

    if(c == '?') {
        for(const string &s:dp) {
            // only keep the last 5 character
            if(check_next(s, '0')) {
                string tmp = s + '0';
                tmp.size() == 6 ? dp_next.insert(tmp.substr(1)) : dp_next.insert(tmp);
            }

            // only keep the last 5 character
            if(check_next(s, '1')) {
                string tmp = s + '1';
                tmp.size() == 6 ? dp_next.insert(tmp.substr(1)) : dp_next.insert(tmp);
            }
        }
    } else {
        for(const string &s:dp) {
            // only keep the last 5 character
            if(check_next(s, c)) {
                string tmp = s + c;
                tmp.size() == 6 ? dp_next.insert(tmp.substr(1)) : dp_next.insert(tmp);
            }
        }
    }
    return dp_next;
}

string sol() {
    if(N < 4) return "POSSIBLE";

    // cache.clear();
    dp.clear();

    // bruteforce generate first 4 chars
    generate(0, "");

    // dp 4->end
    for(int i=4; i<N; ++i) {
        dp = get_next_state(dp, S[i]);
        if(dp.size() == 0) return "IMPOSSIBLE";
    }
    return "POSSIBLE";
}
```

#### Bottomup Bitmask + cache

```Cpp
int N;
string S;


bool is_pal(const string &s) {
    int l = 0;
    int r = s.size() -1;
    while(l < r) {
        if(s[l] != s[r]) return false;
        l += 1;
        r -= 1;
    }
    return true;
}

bool palin_5[(1 << 5) + 3];
bool palin_6[(1 << 6) + 3];
void init() {
    // Generate cache for palin 5
    for(int mask=0;  mask<(1 << 5); ++mask){
        string s = "";
        for(int k=0; k<5; ++k) {
            s = s + (mask & (1 << k) ? '1' : '0');
        }
        palin_5[mask] = is_pal(s);
    }

    // Generate cache for palin 6
    for(int mask=0;  mask<(1 << 6); ++mask){
        string s = "";
        for(int k=0; k<6; ++k) {
            s = s + (mask & (1 << k) ? '1' : '0');
        }
        palin_6[mask] = is_pal(s);
    }
}

int cache[50003][(1 << 6) + 3];
bool get(int idx, int k5, int k6) {
    if(idx >= 5 && palin_5[k5]) return cache[idx][k6] = false;
    if(idx >= 6 && palin_6[k6]) return cache[idx][k6] = false;
    if(idx == N) return true;
    if(cache[idx][k6] != -1) return cache[idx][k6];

    int nk5, nk6;
    if(S[idx] == '?') {
        // 0
        nk5 = (k5 << 1);
        nk5 &= (1 << 5) - 1;
        nk6 = (k6 << 1);
        nk6 &= (1 << 6) - 1;
        bool case_0 = get(idx + 1, nk5, nk6);

        // 1
        nk5 = (k5 << 1) + 1;
        nk5 &= (1 << 5) - 1;
        nk6 = (k6 << 1) + 1;
        nk6 &= (1 << 6) - 1;
        bool case_1 = get(idx + 1, nk5, nk6);
        return case_0 || case_1;
    }
    else if(S[idx] == '0') {
        nk5 = (k5 << 1);
        nk5 &= (1 << 5) - 1;
        nk6 = (k6 << 1);
        nk6 &= (1 << 6) - 1;
        return get(idx + 1, nk5, nk6);
    } else if(S[idx] == '1') {
        nk5 = (k5 << 1) + 1;
        nk5 &= (1 << 5) - 1;
        nk6 = (k6 << 1) + 1;
        nk6 &= (1 << 6) - 1;
        return get(idx + 1, nk5, nk6);
    }
    return false;
}

string sol() {
    memset(palin_5, false, sizeof(palin_5));
    memset(palin_6, false, sizeof(palin_6));
    memset(cache, -1, sizeof(cache));
    init();

    if(get(0,0,0)) return "POSSIBLE";
    return "IMPOSSIBLE";
}
```